### Import and Load Images

In [1]:
import numpy as np
import cv2
import  matplotlib.pyplot as plt
import os
import sys
from tqdm import tqdm_notebook, tnrange
import tqdm
sys.path.append('../Scripts/')
from Scripts.Utility import Utility2
ut = Utility2()

root_dir = os.path.abspath("../")
imagePath = r"F:/CellImagesMed9/whole_cell_kb" #"/Input/Images"
dataimagePath = root_dir + "/Data/Images/Fl"

in init


In [3]:
import  tqdm
def loadImages(path):
    images = []
    imagePaths = os.listdir(path)
    for i, id_ in tqdm_notebook(enumerate(imagePaths),total=len(imagePaths)):
        t = (i + 1)%5
        p = (i + 1)% 15
        if p == 0:
            for j in range(4):
                img = cv2.imread(path + "/" + imagePaths[i-j -1])
        #img = cv2.imread(path + "/" + imagePaths[i])
                images.append(img)
    return images

images = loadImages(imagePath)


#ut.showImages(images, scale=(500,500))
#transmitted, channels = ut.loadImages(imagePath)
#ut.showImages(transmitted, scale=(500,500))
#ut.showImages(channels, scale=(500,500))

C:\anaconda\envs\ImageSegmentation\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


### Apply processing
1. Gabor
2. Histogram Equalization
3. Median Smooth


![Histogram Comparison](/ImageWrite/histogramComparison2.png "Image")

In [4]:
processOutput = []
processOutputCLAHEeq = []
processOutputBaseEq = []

#for ii in tqdm.tqdm.
for i, id_ in tqdm_notebook(enumerate(images),total=len(images)):
#for image in images:
    #smoothed_eq_img_arr, smt_arr, f_img, f_values = ut.processImage(image)

    #processOutput.append(smoothed_eq_img_arr)


    grayImg = ut.grayscaleImages([images[i]]) #([image])
    baseeq = ut.grayscaleImages([ut.histogram_equalization(images[i])])

    processOutputBaseEq.append(baseeq[0])

    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    mg_eq = clahe.apply(grayImg[i]) #0 instead of i before

    processOutputCLAHEeq.append(mg_eq)

ut.writeImages(processOutputCLAHEeq, path="../Input/Images/EqFluor", prefix="fluorEqZ2_", filetype=".jpg", scale=(2048,2048))




#smoothed_eq_img_arr, smt_arr, f_img, f_values = ut.processImage(images[0])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--BFP.tif")
#smoothed_eq_img_arr1, smt_arr1, f_img1, f_values1 = ut.processImage(images[1])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--DsRed.tif")
#smoothed_eq_img_arr2, smt_arr2, f_img2, f_values2 = ut.processImage(images[2])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--eGFP.tif")
#smoothed_eq_img_arr3, smt_arr3, f_img3, f_values3 = ut.processImage(images[3])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--miRFP.tif")




#pairs = ut.pairs_from_array(processOutputCLAHEeq, processOutputBaseEq)
#pairs = ut.pairs_from_array(ut.grayscaleImages(images), pairs)

#ut.showImages(pairs, scale=(1500,500))

In [25]:
reshapedClahe = []

for x in processOutputCLAHEeq:
    V = [x,x,x]
    I = cv2.merge(V)
    reshapedClahe.append(I)
    print(I.shape)


(2048, 2048, 3)
(2048, 2048, 3)
(2048, 2048, 3)
(2048, 2048, 3)
(2048, 2048, 3)


In [28]:
kmean = ut.kmeansOutImages(reshapedClahe, kVal=3)

ut.showImages(kmean, scale=(500,500))

### Apply Median Smoothing

In [13]:
medianSmoothed = ut.medianBlurImages(processOutputCLAHEeq)
pairs = ut.pairs_from_array(processOutputCLAHEeq, medianSmoothed)


ut.showImages(pairs, scale=(1000,500))

### Merge Processed Images (Single Pair)
Gamma = 0.25

Theta = pi/4 && 3*pi/4

From index 1 and 5 of processed images

In [8]:
#smts = [smoothed_eq_img_arr,smoothed_eq_img_arr1, smoothed_eq_img_arr2,smoothed_eq_img_arr3]
#mergedImages = ut.mergeImages(smts, 0,3, 0.5)
mergedImages = []

for x in processOutput:
    mergedImage = cv2.addWeighted(x[1], 0.5, x[5], .5, 0)
    mergedImages.append(mergedImage)
    
#ut.showImages(mergedImages, scale=(500,500))

pairs = ut.pairs_from_array(processOutputCLAHEeq, mergedImages)
pairs = ut.pairs_from_array(ut.grayscaleImages(images), pairs)

ut.showImages(pairs, scale=(1500,500))


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\core\src\matrix_operations.cpp:68: error: (-215:Assertion failed) src[i].dims <= 2 && src[i].rows == src[0].rows && src[i].type() == src[0].type() in function 'cv::hconcat'


### Watershed Process

In [63]:
#mergedImages = [mergedImage, mergedImage1, mergedImage2, mergedImage3]
#GrayMergedArr = ut.grayscaleImages(mergedImages)

ThresholdArr = ut.thresholdImages(processOutputCLAHEeq, invert=False)
pairs = ut.pairs_from_array(processOutputCLAHEeq, ThresholdArr)


ut.showImages(pairs, scale=(1000,500))
#kmeansApplied = ut.kmeansOutImages(ThresholdArr, 5)


In [80]:
i = 2

#numLabels, markers = ut.waterShedImages([ThresholdArr[0]])

openings = ut.morphologyImages(ThresholdArr)
        #ut.showImages(thresholded, scale=(500, 500))

sure_bgs = ut.dilateImages(openings)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(openings[i], cv2.DIST_L2, 5)
dist_transform = cv2.normalize(dist_transform, dist_transform, 0, 1.0, cv2.NORM_MINMAX)
ret, sure_fg = cv2.threshold(dist_transform, 0.1 * dist_transform.max(), 255, 0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bgs[i], sure_fg)

num_labels, labels = cv2.connectedComponents(sure_fg)

label_hue = np.uint8(179 * labels/np.max(labels))
blank_ch = 255 * np.ones_like(label_hue)
labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

#markers = markers + 1
# Now, mark the region of unknown with zero
labeled_img[label_hue == 0] = 0

#markers = cv2.watershed(images[0], markers)
I = processOutputCLAHEeq[i]
#images[0][markers == -1] = [255, 0, 0]
pairs = ut.pairs_from_array([cv2.merge([I,I,I])], [labeled_img])


grayImg = ut.grayscaleImages(images)
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))

hist_img = clahe.apply(grayImg[4])

I = cv2.merge([hist_img,hist_img,hist_img])

mergedImage = cv2.addWeighted(I, 0.5, labeled_img, .5, 0)

pairs = ut.pairs_from_array(pairs,[mergedImage])

ut.writeImages(pairs, prefix="YesMorphology" + str(i))

ut.showImages(pairs, scale=(1500,500))